# **Elaborazione di Immagini Mediche**
## Vessel Wall Segmentation Challenge - A.A. 2021/22 

### Script di Preprocessing

Rigazio Sofia, Roccaro Lucia, Romano Anastasio, Ruzzante Elena

Collegamento a Google Drive e installazione delle librerie necessarie

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install tensorflow==2.1.0
!pip install keras==2.3.1
!pip install plotly==5.3.1



Importazione delle librerie necessarie



In [ ]:
import os
import random
import numpy as np
import plotly.express as px
import pickle
import math

from matplotlib import pyplot as plt
from tqdm import tqdm
from skimage.io import imread
from skimage.transform import resize
from skimage.segmentation import mark_boundaries

from keras.utils.np_utils import to_categorical

from skimage.measure import regionprops
import tensorflow as tf
import json

Definizione delle directory e dei path

In [ ]:
current_dir = '/content/drive/MyDrive/Colab Notebooks/EIM/VesselWallSegmentationChallenge'
dataset_name = '3D_dataset'

TRAIN_path = os.path.join(current_dir,dataset_name,'TRAIN')
VAL_path = os.path.join(current_dir,dataset_name,'VALIDATION')

JSON_path = os.path.join(current_dir,'PICKLE_and_JSON')
PICKLE_ICA_path = os.path.join(JSON_path,'ICA')
PICKLE_ECA_path = os.path.join(JSON_path,'ECA')

carotid_types = ['ICA','ECA']
arts = ['ICAL','ECAL','ICAR','ECAR']

# Creazione della cartelle che conterranno file utili da passare tra gli script
for carotid_type in carotid_types:
  try:
    os.mkdir(eval(f'PICKLE_{carotid_type}_path'))
  except:
    pass

Estrazione della lista di volumi di Training e Validation set e load dei dizionari contenenti le informazioni sulle slice

In [ ]:
train_folders_ICA = []
train_folders_ECA = []
val_folders_ICA = []
val_folders_ECA = []
for arti in arts:
  # Training set
  for casei in os.listdir(TRAIN_path):
    cpath = os.path.join(TRAIN_path, casei, arti)
    if arti[0:3] == carotid_types[0]: # ICA
      train_folders_ICA.append(cpath)
    else: # ECA
      train_folders_ECA.append(cpath)

  # Validation set
  for casei in os.listdir(VAL_path):
    cpath = os.path.join(VAL_path, casei, arti)
    if arti[0:3] == carotid_types[0]: # ICA
      val_folders_ICA.append(cpath)
    else: # ECA
      val_folders_ECA.append(cpath)

# load dei dizionari contenenti le informazioni sulle immagini
with open(os.path.join(JSON_path,'labels_TRAIN.json'),"r") as f:
  labels_TRAIN = json.load(f)
  f.close
with open(os.path.join(JSON_path,'labels_VAL.json'),"r") as f:
  labels_VAL = json.load(f)
  f.close

Settaggio dei parametri 

In [ ]:
# Definiamo i parametri
typical_shape = labels_VAL[list(labels_VAL.keys())[0]]['original_shape']
IMG_WIDTH = typical_shape[1] # la larghezza sono le colonne
IMG_HEIGHT = typical_shape[0] # l'altezza sono le righe
NUM_CLASSES = 3 # vogliamo segmentare tre classi per ogni rete: lume, wall e background
IMG_CHANNELS = 1 # Le immagini fornite sono grayscale
# il numero di slice è variabile -> non lo definiamo
print(f'Dimensioni originali tipiche: {typical_shape}')

Visualizzazione del contorno dell'oggetto su una slice dell'immagine originale

In [ ]:
# Troviamo la prima cartella esistente dalla lista di cartelle ICA
chosen_folder = None 
i = 0
while chosen_folder == None:
  try:
    volume = imread(os.path.join(train_folders_ICA[i], 'image.tiff'))
    chosen_folder = train_folders_ICA[i]
  except:
    pass
  i = i+1
lumen_mask = imread(os.path.join(chosen_folder, 'lumen_mask.tiff'))
wall_mask = imread(os.path.join(chosen_folder, 'wall_mask.tiff'))

# Estrazione della prima slice
slice_chosen = 1
slice_1 = volume[slice_chosen-1,:,:]
wall_mask_1 = wall_mask[slice_chosen-1,:,:]
lumen_mask_1 = lumen_mask[slice_chosen-1,:,:]

# Plot a video dell'immagine originale e della corrispondente segmentazione manuale
fig = plt.figure(figsize=(25,15))
ax1 = fig.add_subplot(3,1,1)
ax1.imshow(slice_1,cmap=plt.cm.gray), ax1.set_title('Original image')

ax2 = fig.add_subplot(3,1,2)
ax2.imshow(wall_mask_1-lumen_mask_1,cmap=plt.cm.gray), ax2.set_title('Manual segmentation')    

ax3 = fig.add_subplot(3,1,3)
ax3.imshow(mark_boundaries(slice_1,(wall_mask_1-lumen_mask_1).astype(np.uint8),color=(1,0,0))), ax3.set_title('Original image + Manual segmentation contour') 
# NOTA: per utilizzare la funzione mark_boundaries è necessario convertire la maschera in formato int 

Creazione delle matrici contenenti immagini e maschere

In [ ]:
def flip_and_crop(image,mask,side):
# flip left-right se è lato sinistro (a destra nell'immagine)  
  if side == 'L':
    image = np.fliplr(image)
    mask = np.fliplr(mask)

  # consideriamo solo la metà sinistra dell'immagine (che contiene ICAL/ECAL flippate o ICAR/ECAR)
  half_width = int(image.shape[1]/2)
  image = image[:,0:half_width]
  mask = mask[:,0:half_width,:]
  
  # controllo sulle dimensioni delle immagini
  if image.shape != (IMG_HEIGHT,int(IMG_WIDTH/2)): #(100,360)
    # effettuiamo un resize with pad per mantenere l'aspect ratio delle immagini 
    # e delle maschere
    image = np.expand_dims(image,axis = 2)
    image = np.squeeze(tf.image.resize_with_pad(image,IMG_HEIGHT,int(IMG_WIDTH/2), antialias=True).numpy())
    mask = tf.image.resize_with_pad(mask,IMG_HEIGHT,int(IMG_WIDTH/2), antialias=True).numpy() 
    # reimpostiamo i valori dei pixel coinvolti nel resize a 0 e 1: i pixel ai 
    # bordi delle segmentazioni potrebbero essersi sfumati
    soglia = 0.5
    mask[mask < soglia] = 0.0
    mask[mask >= soglia] = 1.0

    # Spostiamo le colonne aggiunte con lo zero padding a sinistra dell'immagine
    # con lo zero padding vengono inserite 80 colonne nere per lato
    to_shift = 80  
    image = np.roll(image, to_shift, axis = 1)
    mask = np.roll(mask, to_shift, axis = 1)

  return image, mask

In [ ]:
def create_matrix (list_folders,n_images,labels):
  # inizializzazioni
  X = np.zeros((n_images,IMG_HEIGHT,int(IMG_WIDTH/2)),dtype = np.uint8)
  Y = np.zeros((n_images,IMG_HEIGHT,int(IMG_WIDTH/2),NUM_CLASSES),dtype = np.float32)
  names = [] # lista che conterrà i nomi delle immagini originali

  slice_counter = 0 # contatore del numero di slice
  for v_subj in tqdm(list_folders, total = len(list_folders)):
    subject_ID = v_subj.split('/')[-2]
    arti = v_subj.split('/')[-1]
    if os.path.exists(v_subj) == True: # se la cartella esiste (ovvero se abbiamo le segmentazioni)
      # Lettura dell'immagine e delle maschere
      volume = imread(os.path.join(v_subj, 'image.tiff'))
      wall_mask = imread(os.path.join(v_subj, 'wall_mask.tiff'))
      lumen_mask = imread(os.path.join(v_subj, 'lumen_mask.tiff'))

      # consideriamo una slice alla volta
      for i in range(volume.shape[0]):
        slice_counter = slice_counter + 1 # incrementiamo il contatore del numero di slice
        
        # Conversione della maschera in dato categorico
        mask_cat = to_categorical(wall_mask[i]+lumen_mask[i], num_classes=NUM_CLASSES, dtype='float32')
        # mask_cat[:,:,0] = background, mask_cat[:,:,1] = wall, mask_cat[:,:,2] = lumen
        
        # Preprocessing
        arti = v_subj.split("/")[-1] # [ICAL, ICAR, ECAL, ECAR]
        [volume_slice,mask_slice] = flip_and_crop(volume[i],mask_cat,arti[-1])
        # creiamo un'unica maschera sommando le maschere di lume e wall, in questo modo avremo:
        # background con valore 0, parete con valore 1, lume con valore 2

        # inserimento di immagine e maschera nella matrice
        X[slice_counter-1] = volume_slice
        Y[slice_counter-1] = mask_slice

        names.append(arti+'_'+labels[subject_ID][arti+'_original']['name'][i])

  return X, Y, names

In [ ]:
# numero di slice disponibili
for carotid_type in carotid_types:
  # Training set
  slice_list = [labels_TRAIN[casei][f"{carotid_type}L_original"]["number"] for casei in os.listdir(TRAIN_path)]
  n_slices_L_train = sum([len(slice_list[i]) for i in range(len(slice_list))])
  slice_list = [labels_TRAIN[casei][f"{carotid_type}R_original"]["number"] for casei in os.listdir(TRAIN_path)]
  n_slices_R_train = sum([len(slice_list[i]) for i in range(len(slice_list))])
  # Validation set
  slice_list = [labels_VAL[casei][f"{carotid_type}L_original"]["number"] for casei in os.listdir(VAL_path)]
  n_slices_L_val = sum([len(slice_list[i]) for i in range(len(slice_list))])
  slice_list = [labels_VAL[casei][f"{carotid_type}R_original"]["number"] for casei in os.listdir(VAL_path)]
  n_slices_R_val = sum([len(slice_list[i]) for i in range(len(slice_list))])
  
  if carotid_type == 'ICA':
    n_images_train_ICA = n_slices_L_train + n_slices_R_train
    n_images_val_ICA = n_slices_L_val + n_slices_R_val
  else:
    n_images_train_ECA = n_slices_L_train + n_slices_R_train
    n_images_val_ECA = n_slices_L_val + n_slices_R_val

# creazione delle matrici
# Training Set
print('Reading images - Training set')
print('ICA')
[X_train_ICA,Y_train_ICA,names_train_ICA] = create_matrix(train_folders_ICA,n_images_train_ICA,labels_TRAIN)
print('ECA')
[X_train_ECA,Y_train_ECA,names_train_ECA] = create_matrix(train_folders_ECA,n_images_train_ECA,labels_TRAIN)

# Validation Set
print('\nReading images - Validation set')
print('ICA')
[X_val_ICA,Y_val_ICA,names_val_ICA] = create_matrix(val_folders_ICA,n_images_val_ICA,labels_VAL)
print('ECA')
[X_val_ECA,Y_val_ECA,names_val_ECA] = create_matrix(val_folders_ECA,n_images_val_ECA,labels_VAL)

Identificazione delle soglie per la rimozione delle biforcazioni

In [ ]:
# numero totale di slice appartenenti al Training set
total_slices = X_train_ICA.shape[0] + X_train_ECA.shape[0]

# inizializziamo le proprietà 
# roundness come vettori di 1 e area come vettore di 0
roundness_wall = np.ones((total_slices,1),dtype = np.float32)
roundness_lumen = np.ones((total_slices,1),dtype = np.float32)
area_wall = np.zeros((total_slices,1),dtype = np.uint32)
area_lumen = np.zeros((total_slices,1),dtype = np.uint32)

for n_slice in range(total_slices):
  if n_slice < X_train_ICA.shape[0]:
    wall_mask = Y_train_ICA[n_slice,:,:,1] + Y_train_ICA[n_slice,:,:,2]
    lumen_mask = Y_train_ICA[n_slice,:,:,2]
  else:
    wall_mask = Y_train_ECA[n_slice-X_train_ICA.shape[0],:,:,1] + Y_train_ECA[n_slice-X_train_ICA.shape[0],:,:,2]
    lumen_mask = Y_train_ECA[n_slice-X_train_ICA.shape[0],:,:,2]

  # lumen
  properties = regionprops(lumen_mask.astype(np.uint8), cache=False)
  # se le maschere sono nere, la funzione regionprops non calcola le proprietà
  if len(properties): 
    if properties[0].major_axis_length == 0:
      # se ci sono maschere con lume composto da un solo punto (l'asse maggiore
      # in questo caso ha lunghezza pari a 0) si tratta di errori: impostiamo 
      # quindi le roundness pari a 0 in modo da eliminare le maschere
      roundness_lumen[n_slice] = 0
      roundness_wall[n_slice] = 0
    else:
      area_lumen[n_slice] = lumen_mask.sum()
      roundness_lumen[n_slice] = 4*area_lumen[n_slice]/(math.pi*properties[0].major_axis_length**2)
      # wall
      area_wall[n_slice] = wall_mask.sum()
      properties = regionprops(wall_mask.astype(np.uint8), cache=False)
      roundness_wall[n_slice] = 4*area_wall[n_slice]/(math.pi*properties[0].major_axis_length**2)

# visualizzaizone degli istogrammi
fig = plt.figure(figsize=(20, 5))
ax1 = plt.subplot(1, 3, 1)
ax1.hist(roundness_lumen, bins=100, range=(0,1))
ax1.set_title('Roundness lumen')

ax2 = plt.subplot(1, 3, 2)
ax2.hist(roundness_wall, bins=100, range=(0,1))
ax2.set_title('Roundness wall')

ax3 = plt.subplot(1, 3, 3)
ax3.hist(area_lumen, bins=100)
ax3.set_title('Area lumen')
fig.show()

Funzione per la rimozione automatica delle slice che contengono biforcazioni

In [ ]:
def rimozione_biforcazioni(X,Y,names):
  # eliminazione automatica delle biforcazioni

  removed_slices = [] # nomi delle slice rimosse
  # matrici di immagini e maschere rimosse, utili per la visualizzazione
  X_bif = np.zeros((1,IMG_HEIGHT,int(IMG_WIDTH/2)),dtype = np.uint8)
  Y_bif = np.zeros((1,IMG_HEIGHT,int(IMG_WIDTH/2),NUM_CLASSES),dtype = np.float32)

  n_slice = 0
  while n_slice < Y.shape[0]:
    wall_mask = Y[n_slice,:,:,1] + Y[n_slice,:,:,2]
    lumen_mask = Y[n_slice,:,:,2]

    # lumen
    properties = regionprops(lumen_mask.astype(np.uint8), cache=False)
    # se le maschere sono nere, la funzione regionprops non calcola le proprietà
    if len(properties): 
      if properties[0].major_axis_length == 0:
        # se ci sono maschere con lume composto da un solo punto (l'asse 
        # maggiore in questo caso ha lunghezza pari a 0) si tratta di errori 
        # nelle segmentazioni manuali: impostiamo quindi le roundness pari a 0 e
        # l'area maggiore della soglia in modo da eliminare le maschere
        roundness_lumen = 0
        roundness_wall = 0
        area_lumen = soglia_area_lumen + 1
      else:
        area_lumen = lumen_mask.sum()
        roundness_lumen = 4*area_lumen/(math.pi*properties[0].major_axis_length**2)
        # wall
        area_wall = wall_mask.sum()
        properties = regionprops(wall_mask.astype(np.uint8), cache=False)
        roundness_wall = 4*area_wall/(math.pi*properties[0].major_axis_length**2)
    else:
      # se le maschere sono nere (quelle introdotte per il training della rete), 
      # impostiamo dei valori delle proprietà in modo che le slice non vengano 
      # eliminate
      roundness_lumen = 1
      roundness_wall = 1
      area_lumen = 0

    # rimozione della maschera e della rispettiva immagine dalle matrici:
    # effettuiamo il controllo sia su wall che su lumen. se solo wall o solo 
    # lumen ha forma allungata, allora non ci troviamo in corrispondenza di una 
    # biforcazione. la forma non circolare potrebbe essere dovuta ad esempio 
    # alla presenza di una placca
    if roundness_wall < soglia_wall and roundness_lumen < soglia_lumen and area_lumen > soglia_area_lumen:
      # concateniamo la slice rimossa alle matrici di visualizzazione
      X_bif = np.vstack((X_bif,np.expand_dims(X[n_slice],axis=0)))
      Y_bif = np.vstack((Y_bif,np.expand_dims(Y[n_slice],axis=0)))
      # concateniamo il nome della slice rimossa alla lista di nomi
      removed_slices.append(names.pop(n_slice))
      # rimuoviamo la slice dalle matrici X e Y
      X = np.delete(X,n_slice,axis=0)
      Y = np.delete(Y,n_slice,axis=0)
      
    else:
      n_slice = n_slice+1

  # eliminiamo la prima slice dalle matrici ottenute con vstack (contiene zeri)   
  X_bif = np.delete(X_bif,0,axis=0)
  Y_bif = np.delete(Y_bif,0,axis=0)

  return X, Y, names, removed_slices, X_bif, Y_bif

In [ ]:
# definizione delle soglie in base agli istogrammi
soglia_wall = 0.7
soglia_lumen = 0.65
soglia_area_lumen = 245

# richiamo della funzione per eliminare le biforcazioni
# ICA
X_train_ICA,Y_train_ICA,names_train_ICA,removed_slices_train_ICA,X_bif_train_ICA,Y_bif_train_ICA = rimozione_biforcazioni(X_train_ICA,Y_train_ICA,names_train_ICA)
X_val_ICA,Y_val_ICA,names_val_ICA,removed_slices_val_ICA,X_bif_val_ICA,Y_bif_val_ICA = rimozione_biforcazioni(X_val_ICA,Y_val_ICA,names_val_ICA)
# ECA
X_train_ECA,Y_train_ECA,names_train_ECA,removed_slices_train_ECA,X_bif_train_ECA,Y_bif_train_ECA = rimozione_biforcazioni(X_train_ECA,Y_train_ECA,names_train_ECA)
X_val_ECA,Y_val_ECA,names_val_ECA,removed_slices_val_ECA,X_bif_val_ECA,Y_bif_val_ECA = rimozione_biforcazioni(X_val_ECA,Y_val_ECA,names_val_ECA)

print('Numero di slice identificate come biforcazioni e rimosse:')
print(f'ICA\tTraining set: {len(removed_slices_train_ICA)}')
print(f'\tValidation set: {len(removed_slices_val_ICA)}')
print(f'ECA\tTraining set: {len(removed_slices_train_ECA)}')
print(f'\tValidation set: {len(removed_slices_val_ECA)}')

# aggiornamento del numero di slice rimanenti
n_images_train_ICA = X_train_ICA.shape[0]
n_images_val_ICA = X_val_ICA.shape[0]
n_images_train_ECA = X_train_ECA.shape[0]
n_images_val_ECA = X_val_ECA.shape[0]

Visualizzazione degli istogrammi con le soglie scelte evidenziate

In [ ]:
fig = plt.figure(figsize=(20, 5))
ax1 = plt.subplot(1, 3, 1)
ax1.hist(roundness_lumen, bins=100, range=(0,1))
ax1.axvline(soglia_lumen, color='r', linestyle='dashed', linewidth=1)
ax1.set_title('Roundness lumen')

ax2 = plt.subplot(1, 3, 2)
ax2.hist(roundness_wall, bins=100, range=(0,1))
ax2.axvline(soglia_wall, color='r', linestyle='dashed', linewidth=1)
ax2.set_title('Roundness wall')

ax3 = plt.subplot(1, 3, 3)
ax3.hist(area_lumen, bins=100)
ax3.axvline(soglia_area_lumen, color='r', linestyle='dashed', linewidth=1)
ax3.set_title('Area lumen')
fig.show()

Visualizzazione delle slice rimosse dalle matrici ICA

In [ ]:
# Training set
Y_vis = np.zeros((len(removed_slices_train_ICA),IMG_HEIGHT, int(IMG_WIDTH/2), 3), dtype = np.float32)
for i in range(len(removed_slices_train_ICA)):
  Y_vis[i] = mark_boundaries(X_bif_train_ICA[i],Y_bif_train_ICA[i,:,:,1].astype(np.uint8))
fig = px.imshow(Y_vis, animation_frame=0, labels=dict(animation_frame="slice"))
for i, frame in enumerate(fig.frames):
  carotide = removed_slices_train_ICA[i].split('_')[0]
  slice_name = removed_slices_train_ICA[i].split('_')[1]
  frame.layout.title = f'Carotide: {carotide} - Nome slice: {slice_name}'
fig.show()

# Validation set
Y_vis = np.zeros((len(removed_slices_val_ICA),IMG_HEIGHT, int(IMG_WIDTH/2), 3), dtype = np.float32)
for i in range(len(removed_slices_val_ICA)):
  Y_vis[i] = mark_boundaries(X_bif_val_ICA[i],Y_bif_val_ICA[i,:,:,1].astype(np.uint8))
fig = px.imshow(Y_vis, animation_frame=0, labels=dict(animation_frame="slice"))
for i, frame in enumerate(fig.frames):
  carotide = removed_slices_val_ICA[i].split('_')[0]
  slice_name = removed_slices_val_ICA[i].split('_')[1]
  frame.layout.title = f'Carotide: {carotide} - Nome slice: {slice_name}'
fig.show()

Rimozione dai dizionari delle slice rimosse in questa fase e salvataggio dei dizionari aggiornati

In [ ]:
for carotid_type in carotid_types:
  # Training set
  for rem_slice in eval(f'removed_slices_train_{carotid_type}'):
    arti = rem_slice.split('_')[0]
    name = rem_slice.split('_')[1]
    for v_subj in labels_TRAIN:
      if name in labels_TRAIN[v_subj][arti]['name']:
        index = labels_TRAIN[v_subj][arti]['name'].index(name)
        labels_TRAIN[v_subj][arti]['name'].pop(index)
        labels_TRAIN[v_subj][arti]['number'].pop(index)

  # Validation set
  for rem_slice in eval(f'removed_slices_val_{carotid_type}'):
    arti = rem_slice.split('_')[0]
    name = rem_slice.split('_')[1]
    for v_subj in labels_VAL:
      if name in labels_VAL[v_subj][arti]['name']:
        index = labels_VAL[v_subj][arti]['name'].index(name)
        labels_VAL[v_subj][arti]['name'].pop(index)
        labels_VAL[v_subj][arti]['number'].pop(index)

# salvataggio dei dizionai aggiornati
with open(os.path.join(JSON_path,'labels_TRAIN.json'),"w") as f:
  json.dump(labels_TRAIN,f,indent = 4)
  f.close
with open(os.path.join(JSON_path,'labels_VAL.json'),"w") as f:
  json.dump(labels_VAL,f,indent = 4)
  f.close

Estrazione della ROI - ICA

In [ ]:
# cerchiamo una ROI nelle immagini, identificata come il minimo rettangolo 
# contenente tutte le segmentazioni manuali delle immagini del Training set
Y_sum = np.sum(Y_train_ICA[:,:,:,1]+Y_train_ICA[:,:,:,2],axis = 0)
Y_sum[Y_sum>0] = 1
fig = px.imshow(Y_sum)
fig.show()

limits = np.where(Y_sum>0)
row_ICA = [limits[0].min(),limits[0].max()]
col_ICA = [limits[1].min(),limits[1].max()] # larghezza 105

# Per essere sicuri di comprendere anche tutti i casi del Test e Validation set,
# aumentiamo le dimensioni della ROI ottenuta del 10% per lato
aumento_dim = 0.1
row_ICA[0] = int(min(abs(row_ICA[0]-round(aumento_dim*(row_ICA[1]-row_ICA[0]))),0))
row_ICA[1] = int(min(IMG_HEIGHT-1, row_ICA[1]+round(aumento_dim*(row_ICA[1]-row_ICA[0]))))

col_ICA[0] = col_ICA[0] - round(aumento_dim*(col_ICA[1]-col_ICA[0]))
col_ICA[1] = int(min(IMG_WIDTH/2-1, col_ICA[1]+round(aumento_dim*(col_ICA[1]-col_ICA[0])))) 

# definiamo le dimensioni della ROI
ROI_HEIGHT_ICA = row_ICA[1]-row_ICA[0]+1 # 100
ROI_WIDTH_ICA = col_ICA[1]-col_ICA[0]+1 # 126
print(f'ROI ICA estratta dal Training set\nrighe:{row_ICA}\ncolonne:{col_ICA}')

Verifica della dimensione della ROI ICA sul Validation set

In [ ]:
# cerchiamo una ROI nelle immagini, identificata come il minimo rettangolo 
# contenente tutte le segmentazioni manuali delle immagini del Training set
Y_sum = np.sum(Y_val_ICA[:,:,:,1]+Y_val_ICA[:,:,:,2],axis = 0)
Y_sum[Y_sum>0] = 1
fig = px.imshow(Y_sum)
fig.show()

limits = np.where(Y_sum>0)
row_v = [limits[0].min(),limits[0].max()]
col_v = [limits[1].min(),limits[1].max()]
print(f'ROI estratta dal Validation set\nrighe:{row_v}\ncolonne:{col_v}')

if col_v[0]< col_ICA[0] or col_v[1]>col_ICA[1]:
  print('Attenzione! Controllare le dimensioni della ROI ICA (Val > Train)')
else:
  print('ROI OK')

Estrazione della ROI - ECA

In [ ]:
# cerchiamo una ROI nelle immagini, identificata come il minimo rettangolo 
# contenente tutte le segmentazioni manuali delle immagini del Training set
Y_sum = np.sum(Y_train_ECA[:,:,:,1]+Y_train_ECA[:,:,:,2],axis = 0)
Y_sum[Y_sum>0] = 1
fig = px.imshow(Y_sum)
fig.show()

limits = np.where(Y_sum>0)
row_ECA = [limits[0].min(),limits[0].max()]
col_ECA = [limits[1].min(),limits[1].max()]

# Per essere sicuri di comprendere anche tutti i casi del Test e Validation set,
# aumentiamo le dimensioni della ROI ottenuta del 15% per lato
aumento_dim = 0.1
row_ECA[0] = int(min(abs(row_ECA[0]-round(aumento_dim*(row_ECA[1]-row_ECA[0]))),0))
row_ECA[1] = int(min(IMG_HEIGHT-1, row_ECA[1]+round(aumento_dim*(row_ECA[1]-row_ECA[0]))))

col_ECA[0] = col_ECA[0] - round(aumento_dim*(col_ECA[1]-col_ECA[0]))
col_ECA[1] = int(min(IMG_WIDTH/2-1, col_ECA[1]+round(aumento_dim*(col_ECA[1]-col_ECA[0]))))

# definiamo le dimensioni della ROI
ROI_HEIGHT_ECA = row_ECA[1]-row_ECA[0]+1 # 98
ROI_WIDTH_ECA = col_ECA[1]-col_ECA[0]+1 # 112
print(f'ROI ECA estratta dal Training set\nrighe:{row_ECA}\ncolonne:{col_ICA}')

Verifica della dimensione della ROI ECA sul Validation set

In [ ]:
# cerchiamo una ROI nelle immagini, identificata come il minimo rettangolo 
# contenente tutte le segmentazioni manuali delle immagini del Training set
Y_sum = np.sum(Y_val_ECA[:,:,:,1]+Y_val_ECA[:,:,:,2],axis = 0)
Y_sum[Y_sum>0] = 1
fig = px.imshow(Y_sum)
fig.show()

limits = np.where(Y_sum>0)
row_v = [limits[0].min(),limits[0].max()]
col_v = [limits[1].min(),limits[1].max()]
print(f'ROI estratta dal Validation set\nrighe:{row_v}\ncolonne:{col_v}')

if col_v[0]< col_ECA[0] or col_v[1]>col_ECA[1]:
  print('Attenzione! Controllare le dimensioni della ROI ECA (Val > Train)')
else:
  print('ROI OK')

Creazione di sottomatrici X e Y contenenti solamente le ROI delle immagini e delle maschere

In [ ]:
# Training set
X_train_ICA = X_train_ICA[:,row_ICA[0]:row_ICA[1]+1,col_ICA[0]:col_ICA[1]+1]
Y_train_ICA = Y_train_ICA[:,row_ICA[0]:row_ICA[1]+1,col_ICA[0]:col_ICA[1]+1,:]
X_train_ECA = X_train_ECA[:,row_ECA[0]:row_ECA[1]+1,col_ECA[0]:col_ECA[1]+1]
Y_train_ECA = Y_train_ECA[:,row_ECA[0]:row_ECA[1]+1,col_ECA[0]:col_ECA[1]+1,:]

# Validation set
X_val_ICA = X_val_ICA[:,row_ICA[0]:row_ICA[1]+1,col_ICA[0]:col_ICA[1]+1]
Y_val_ICA = Y_val_ICA[:,row_ICA[0]:row_ICA[1]+1,col_ICA[0]:col_ICA[1]+1,:]
X_val_ECA = X_val_ECA[:,row_ECA[0]:row_ECA[1]+1,col_ECA[0]:col_ECA[1]+1]
Y_val_ECA = Y_val_ECA[:,row_ECA[0]:row_ECA[1]+1,col_ECA[0]:col_ECA[1]+1,:]

Verifica di corretta creazione delle matrici

In [ ]:
# numero di slice da visualizzare
n_slices = 30

# Training Set
index = random.randint(0,n_images_train_ICA-(n_slices+1))

Y_vis = np.zeros_like(Y_train_ICA[index:index+n_slices])
for n_slice in range(n_slices):
  Y_vis[n_slice] = mark_boundaries(X_train_ICA[index+n_slice],Y_train_ICA[index+n_slice,:,:,1].astype(np.uint8))

fig = px.imshow(Y_vis, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()

# Validation Set
index = random.randint(0,n_images_val_ICA-(n_slices+1))

Y_vis = np.zeros_like(Y_val_ICA[index:index+n_slices])
for n_slice in range(n_slices):
  Y_vis[n_slice] = mark_boundaries(X_val_ICA[index+n_slice],Y_val_ICA[index+n_slice,:,:,1].astype(np.uint8))

fig = px.imshow(Y_vis, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()

Preparazione delle matrici per la rete

In [ ]:
# Passiamo da dimensioni (x,x,x) a dimensioni (x,x,x,1) come richiesto dalla rete
X_train_ICA = np.expand_dims(X_train_ICA, axis=3)
X_val_ICA = np.expand_dims(X_val_ICA, axis=3)

X_train_ECA = np.expand_dims(X_train_ECA, axis=3)
X_val_ECA = np.expand_dims(X_val_ECA, axis=3)

# diamo in input alla rete delle immagini quadrate di dimensioni multiple di 32 
# -> troviamo il minimo multiplo di 32 superiore alla maggiore larghezza attuale
# NOTA: impostiamo la stessa dimensione per le immagini delle carotidi ICA ed
# ECA da fornire alla rete poiché alleniamo la rete ECA partendo dai pesi della
# rete ICA
bigger_width = max(ROI_WIDTH_ECA,ROI_WIDTH_ICA)
NET_IMG_WIDTH = -((-bigger_width) // 32) * 32 # 160
NET_IMG_HEIGHT = -((-bigger_width) // 32) * 32 # 160
# input_shape: shape of input data/image ``(H, W, C)``, in general case you do 
# not need to set ``H`` and ``W`` shapes, just pass ``(None, None, C)`` to make 
# your model be able to process images af any size, but ``H`` and ``W`` of input
# images should be divisible by factor ``32``

# Zero padding delle immagini fino alla dimensione (NET_IMG_HEIGHT,NET_IMG_WIDTH)
X_train_ICA = tf.image.pad_to_bounding_box(X_train_ICA, 0, 0, NET_IMG_HEIGHT, NET_IMG_WIDTH).numpy()
Y_train_ICA = tf.image.pad_to_bounding_box(Y_train_ICA, 0, 0, NET_IMG_HEIGHT, NET_IMG_WIDTH).numpy()
X_val_ICA = tf.image.pad_to_bounding_box(X_val_ICA, 0, 0, NET_IMG_HEIGHT, NET_IMG_WIDTH).numpy()
Y_val_ICA = tf.image.pad_to_bounding_box(Y_val_ICA, 0, 0, NET_IMG_HEIGHT, NET_IMG_WIDTH).numpy()

X_train_ECA = tf.image.pad_to_bounding_box(X_train_ECA, 0, 0, NET_IMG_HEIGHT, NET_IMG_WIDTH).numpy()
Y_train_ECA = tf.image.pad_to_bounding_box(Y_train_ECA, 0, 0, NET_IMG_HEIGHT, NET_IMG_WIDTH).numpy()
X_val_ECA = tf.image.pad_to_bounding_box(X_val_ECA, 0, 0, NET_IMG_HEIGHT, NET_IMG_WIDTH).numpy()
Y_val_ECA = tf.image.pad_to_bounding_box(Y_val_ECA, 0, 0, NET_IMG_HEIGHT, NET_IMG_WIDTH).numpy()

Salvataggio delle matrici create e delle informazioni sulle ROI e sulle immagini da fornire alla rete

In [ ]:
# ICA
with open(os.path.join(PICKLE_ICA_path,'train_ROI_biforcazioni.pickle'), 'wb') as f:
    pickle.dump([X_train_ICA,Y_train_ICA,names_train_ICA,X_bif_train_ICA,Y_bif_train_ICA], f)

with open(os.path.join(PICKLE_ICA_path,'validation_ROI_biforcazioni.pickle'), 'wb') as f:
    pickle.dump([X_val_ICA,Y_val_ICA,names_val_ICA,X_bif_val_ICA,Y_bif_val_ICA], f)

with open(os.path.join(PICKLE_ICA_path,'ROI_NET_informations.pickle'), 'wb') as f:
    pickle.dump([row_ICA,col_ICA,ROI_HEIGHT_ICA,ROI_WIDTH_ICA,NET_IMG_HEIGHT,NET_IMG_WIDTH], f)

# ECA
with open(os.path.join(PICKLE_ECA_path,'train_ROI_biforcazioni.pickle'), 'wb') as f:
    pickle.dump([X_train_ECA,Y_train_ECA,names_train_ECA,X_bif_train_ECA,Y_bif_train_ECA], f)

with open(os.path.join(PICKLE_ECA_path,'validation_ROI_biforcazioni.pickle'), 'wb') as f:
    pickle.dump([X_val_ECA,Y_val_ECA,names_val_ECA,X_bif_val_ECA,Y_bif_val_ECA], f)

with open(os.path.join(PICKLE_ECA_path,'ROI_NET_informations.pickle'), 'wb') as f:
    pickle.dump([row_ECA,col_ECA,ROI_HEIGHT_ECA,ROI_WIDTH_ECA,NET_IMG_HEIGHT,NET_IMG_WIDTH], f)

Determinazione delle aree da utilizzare in postprocessing

In [ ]:
# Lumen
sum_lumi = [np.sum(Y_train_ICA[i,:,:,2]) for i in range(Y_train_ICA.shape[0])]
sum_lumi_ECA = [np.sum(Y_train_ECA[i,:,:,2]) for i in range(Y_train_ECA.shape[0])]

sum_lumi.extend(sum_lumi_ECA) 
sum_lumi = np.asarray(sum_lumi)

# plot istogrammi
fig = plt.figure(figsize=(15,6))
fig.suptitle('Istogrammi Aree - Training Set', fontsize=16)

ax1 = plt.subplot(2, 2, 1)
plt.hist(sum_lumi,bins=100)
plt.title('Lumen')

ax2 = plt.subplot(2, 2, 2)
plt.hist(sum_lumi[sum_lumi<100],bins=100) 
plt.title('Zoom: Lumen < 100')

# Wall
sum_wall = [np.sum(Y_train_ICA[i,:,:,1]+Y_train_ICA[i,:,:,2]) for i in range(Y_train_ICA.shape[0])]
sum_wall_ECA = [np.sum(Y_train_ECA[i,:,:,1]+Y_train_ECA[i,:,:,2]) for i in range(Y_train_ECA.shape[0])]

sum_wall.extend(sum_lumi_ECA) 
sum_wall = np.asarray(sum_wall)

# plot istogrammi
ax3 = plt.subplot(2, 2, 3) 
plt.hist(sum_wall,bins=100) 
plt.title('Wall')

ax4 = plt.subplot(2, 2, 4)
plt.hist(sum_wall[sum_wall<200],bins=100)
plt.title('Zoom: Wall < 200')

# Escludendo le aree pari a 0 (maschere nere) e poche eccezioni, per i lumi 
# identifichiamo minima area pari a circa 40, mentre per i wall pari a circa 75.
# -> scegliamo quindi le soglie di area per eliminare i piccoli oggetti in post 
# processing (skimage - remove_small_objects) 
# -> per riempire i piccoli buchi (skimage - remove_small_holes) invece, 
# scegliamo soglia pari a 10 sia per lumen che per wall

# inserimento delle soglie scelte nell'immagine
soglia_holes = 10
soglia_obj_wall = 75
soglia_obj_lumen = 40

ax1.axvline(soglia_obj_lumen, color='red', linewidth=1, linestyle='--')
ax2.axvline(soglia_obj_lumen, color='red', linewidth=1, linestyle='--')

ax3.axvline(soglia_obj_wall, color='red', linewidth=1, linestyle='--')
ax4.axvline(soglia_obj_wall, color='red', linewidth=1, linestyle='--')

fig.tight_layout() #tight margins
plt.show()
